# <div align="center">  
# Turn Detection

## Imports

In [2]:
import kielmat


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
import pandas as pd

## Insert Raw IMU Data

In [5]:
# Insert file path
csv_file_path = 'IMUS1.csv'
dataname = 'IMUS1'

## CSV to TSV

In [6]:
import csv 

In [7]:
def csv_to_tsv(input_csv, output_tsv):
    with open(input_csv, 'r', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        with open(output_tsv, 'w', newline='') as tsv_file:
            tsv_writer = csv.writer(tsv_file, delimiter='\t')
            for row in csv_reader:
                tsv_writer.writerow(row)


In [8]:
csv_to_tsv(csv_file_path, f"{dataname}.tsv")

In [9]:
file_path = f"{dataname}.tsv"

## Data Cleaning


In [10]:
sampling_frequency = 128
sampling_rate = 128
tracking_sys = "IMU"
tracked_points = {tracking_sys: ["LowerBack"]}

In [11]:
def read_and_clean_tsv(file_path, new_headers, skiprows=2):
    """
    Reads a TSV file into a DataFrame, skips rows, and assigns new headers.

    Parameters:
        file_path (str): The path to the TSV file.
        new_headers (list): A list of new column headers to assign.
        skiprows (int): The number of rows to skip from the start of the file (default is 2).

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Read the TSV file into a DataFrame with specified headers
    df = pd.read_csv(file_path, sep='\t', skiprows=skiprows, header=1)

    # Assign the new headers to the DataFrame
    df.columns = new_headers

    return df

# Example usage
file_path = 'IMUS1.tsv'
new_headers = [
    'Time', 'Accelerometer_X', 'Accelerometer_Y', 'Accelerometer_Z',
    'Gyroscope_X', 'Gyroscope_Y', 'Gyroscope_Z',
    'Magnetometer_X', 'Magnetometer_Y', 'Magnetometer_Z',
    'Barometer', 'Orientation_S', 'Orientation_X', 'Orientation_Y', 'Orientation_Z'
]

df = read_and_clean_tsv(file_path, new_headers)

# Display the shape and the first few rows of the DataFrame
df.head(5)

,Time,Accelerometer_X,Accelerometer_Y,Accelerometer_Z,Gyroscope_X,Gyroscope_Y,Gyroscope_Z,Magnetometer_X,Magnetometer_Y,Magnetometer_Z,Barometer,Orientation_S,Orientation_X,Orientation_Y,Orientation_Z
0,1.645200e+15,-9.900382,0.768888,1.488947,0.063933,0.033331,0.000101,29.908138,9.080527,15.478364,4.636850e+18,-0.764878,-0.044339,-0.642517,-0.012959
1,1.645200e+15,-9.875965,0.817546,1.549903,0.086359,0.022508,-0.015907,29.908138,9.167147,15.478988,4.636850e+18,-0.764806,-0.044544,-0.642594,-0.012686
2,1.645200e+15,-9.924262,0.817896,1.574577,0.098184,0.016069,-0.031928,29.835429,8.840564,15.588052,4.636850e+18,-0.764749,-0.044744,-0.642655,-0.012336
3,1.645200e+15,-9.900385,0.902713,1.611105,0.108872,0.005420,-0.044767,29.919711,8.607941,15.600971,4.636850e+18,-0.764717,-0.044945,-0.642687,-0.011918
4,1.645200e+15,-9.888087,0.988165,1.575027,0.115312,0.010642,-0.056525,29.920345,8.763737,15.492010,4.636850e+18,-0.764671,-0.045135,-0.642736,-0.011450


In [12]:
def import_df(dataframe,tracked_points=["lowerBack"], sample_rate=128):
    # if we have just one string, put it into a list for consistency 
    if isinstance(tracked_points, str):     
        tracked_points = [tracked_points]
        
    # dictionary to store channels
    channels_dict = {
    "name": [],
    "component": [],
    "type": [],
    "tracked_point": [],
    "units": [],
    "sampling_frequency": [],
    }

    # dictionary for the data 
    data_dict = {}

    # defining sensor types that are consistent with the other import functions 
    sensor_types = ["ACCEL", "GYRO", "MAGN", "BARO", "ORIENT"]
    # just setting units for each t
    units = {
        "ACCEL": "m/s^2",
        "GYRO": "rad/s",
        "MAGN": "µT",
        "BARO": "Pa",
        "ORIENT": "quaternion"
    }

    sample_rate = sample_rate


    for col in df.columns:
            if col == "Time":
                continue

            sensor_type = next((st for st in sensor_types if st in col.upper()), None)
            if not sensor_type:
                continue

            component = col.split('_')[-1]
            data_dict[col] = df[col]

            channels_dict["name"].append(col)
            channels_dict["component"].append(component)
            channels_dict["type"].append(sensor_type)
            channels_dict["tracked_point"].append(tracked_points[0])
            channels_dict["units"].append(units[sensor_type])
            channels_dict["sampling_frequency"].append(sample_rate)

    # Create DataFrame from data_dict
    data = pd.DataFrame(data_dict)

    # Create DataFrame from channels_dict
    channels = pd.DataFrame(channels_dict)
    return data, channels



In [13]:
data, channels = import_df(df, sample_rate=128) 



data

acceldata = data[['Accelerometer_X','Accelerometer_Y','Accelerometer_Z']] # the input argument for the package says it needs 3 cols and only 3 cols
gyrodata = data[['Gyroscope_X','Gyroscope_Y','Gyroscope_Z']]

In [14]:
channels

,name,component,type,tracked_point,units,sampling_frequency
0,Accelerometer_X,X,ACCEL,lowerBack,m/s^2,128
1,Accelerometer_Y,Y,ACCEL,lowerBack,m/s^2,128
2,Accelerometer_Z,Z,ACCEL,lowerBack,m/s^2,128
3,Gyroscope_X,X,GYRO,lowerBack,rad/s,128
4,Gyroscope_Y,Y,GYRO,lowerBack,rad/s,128
5,Gyroscope_Z,Z,GYRO,lowerBack,rad/s,128
6,Magnetometer_X,X,MAGN,lowerBack,µT,128
7,Magnetometer_Y,Y,MAGN,lowerBack,µT,128
8,Magnetometer_Z,Z,MAGN,lowerBack,µT,128
9,Barometer,Barometer,BARO,lowerBack,Pa,128


In [15]:
gyrodata

,Gyroscope_X,Gyroscope_Y,Gyroscope_Z
0,0.063933,0.033331,0.000101
1,0.086359,0.022508,-0.015907
2,0.098184,0.016069,-0.031928
3,0.108872,0.005420,-0.044767
4,0.115312,0.010642,-0.056525
...,...,...,...
56006,-0.038000,0.043341,-0.021978
56007,-0.035574,0.048159,-0.022269
56008,-0.032660,0.053733,-0.023499
56009,-0.030687,0.052176,-0.024404


## Isolate Gyro Z and convert to degrees


In [18]:
# Isolate z
gyro_z = gyrodata["Gyroscope_Z"]

# Convert to degrees per second
conversion_factor = 180 / np.pi
gyro_z_deg_per_s = gyro_z * conversion_factor 

# Time step (1/sampling_rate gives the time between two samples)
dt = 1 / sampling_rate

# Integrate the angular velocity to get the angle (cumulative sum)
gyro_z_deg = np.cumsum(gyro_z_deg_per_s * dt)

# gyro_z_deg now contains the cumulative degrees at each point, assuming 0 degrees at the first point
gyro_z_deg


0          0.000045
1         -0.007075
2         -0.021367
3         -0.041406
4         -0.066708
            ...    
56006    351.198498
56007    351.188530
56008    351.178012
56009    351.167088
56010    351.155410
Name: Gyroscope_Z, Length: 56011, dtype: float64